# $100 S&P500 investment test

Suppose you are an investor with $100 in 2005. Write a python program that calculates the total return (including dividends) if you invested this money into the S&P500. If you could erase any continuous 11 calendar day period from this history in order to get the best returns, what would your new total return be?

## Attempt 1

**Description:**
- Identifies the single 11-day period with the lowest cumulative return and removes it.
- Uses a rolling product of daily returns to find the worst period.
- Recalculates portfolio value after removal.

**Pros:** Simple, easy to understand, and relatively fast.\
**Cons:** Does not account for the timing of losses — removing an early loss vs a late loss can have different compounding effects, which this method ignores.

In [43]:
import yfinance as yf

def remove_worst_n_day_period(df, sv=100, n=11):
    """
    Remove the worst consecutive n-day period from a price history and recalculate portfolio value.

    Parameters:
        df (DataFrame): Price history with 'Close' column.
        sv (float): Starting value of investment (default $100).
        n (int): Number of consecutive days to remove (default 11).

    Returns:
        df_filtered (DataFrame): DataFrame after removing worst period with portfolio value.
        original_total (float): Total portfolio value without removing any days.
        new_total (float): Total portfolio value after removing worst period.
    """
    df = df.copy()
    df["Daily Return"] = df["Close"].pct_change()
    df["Growth Factor"] = 1 + df["Daily Return"]
    
    # Rolling n-day cumulative return
    df["Rolling Return"] = df["Growth Factor"].rolling(window=n).apply(lambda x: x.prod() - 1, raw=True)
    
    # Identify worst n-day period
    worst_end_idx = df["Rolling Return"].idxmin()
    pos = df.index.get_loc(worst_end_idx)
    worst_period_idx = df.iloc[pos - n + 1 : pos + 1].index
    
    # Drop worst period
    df_filtered = df.drop(worst_period_idx)
    
    # Recalculate cumulative portfolio value
    df_filtered["Portfolio Value"] = sv * (1 + df_filtered["Daily Return"]).cumprod()
    original_total = sv * (1 + df["Daily Return"]).cumprod().iloc[- 1]
    new_total = df_filtered["Portfolio Value"].iloc[- 1]
    
    return df_filtered, original_total, new_total

# Example usage
ticker = yf.Ticker("^SPX")
hd = ticker.history(start="2005-01-01", auto_adjust=True)

hd_filtered, original_total, new_total = remove_worst_n_day_period(hd, sv=100, n=11)
print(f"Original total return: ${original_total:.2f}")
print(f"New total return after erasing worst period: ${new_total:.2f}")


Original total return: $529.93
New total return after erasing worst period: $712.59


## Attempt 2

**Description:**
- Iterates through every possible 11-day block, removes it temporarily, and calculates the final portfolio value to determine which block maximises wealth.
- Fully accounts for compounding, because the effect of each block on the final portfolio is measured directly.

**Pros:** Accurate — properly handles timing and compounding.\
**Cons:** Slower, because it repeatedly calculates cumulative portfolio values for almost the entire dataset for each block.

In [41]:
def remove_worst_n_day_period(df, sv=100, n=11):
    """
    Remove the worst consecutive n-day period from a price history and recalculate portfolio value.

    Parameters:
        df (DataFrame): Price history with 'Close' column.
        sv (float): Starting value of investment (default $100).
        n (int): Number of consecutive days to remove (default 11).

    Returns:
        df_filtered (DataFrame): DataFrame after removing worst period with portfolio value.
        original_total (float): Total portfolio value without removing any days.
        new_total (float): Total portfolio value after removing worst period.
    """
    df = df.copy()
    df["Daily Return"] = df["Close"].pct_change()
    df = df.dropna()  # drop first NaN

    best_final_value = 0
    best_block_idx = None

    # Convert index to integer positions for easy slicing
    total_days = len(df)
    
    for start in range(0, total_days - n + 1):
        end = start + n
        temp_df = df.drop(df.index[start:end])
        temp_portfolio = sv * (1 + temp_df["Daily Return"]).cumprod().iloc[-1]
        
        if temp_portfolio > best_final_value:
            best_final_value = temp_portfolio
            best_block_idx = df.index[start:end]

    # Remove the worst block permanently
    df_filtered = df.drop(best_block_idx)
    df_filtered["Portfolio Value"] = sv * (1 + df_filtered["Daily Return"]).cumprod()
    
    # Original portfolio value
    original_total = sv * (1 + df["Daily Return"]).cumprod().iloc[-1]
    new_total = df_filtered["Portfolio Value"].iloc[-1]
    
    return df_filtered, original_total, new_total

# Example usage
ticker = yf.Ticker("^SPX")
hd = ticker.history(start="2005-01-01", auto_adjust=True)

hd_filtered, original_total, new_total = remove_worst_n_day_period(hd, sv=100, n=11)
print(f"Original total return: ${original_total:.2f}")
print(f"New total return after erasing worst period: ${new_total:.2f}")

Original total return: $529.93
New total return after erasing worst period: $712.59


## Attempt 3

**Description:**
- Optimises Attempt 2 by using NumPy arrays and log returns.
- Converts daily returns to log space so that multiplication becomes addition, allowing fast calculation of the effect of removing any 11-day block.
- Iterates through blocks, subtracting the log-sum of each block to efficiently find the one that maximises final portfolio value.

**Pros:**
- Fully compounding-aware, like Attempt 2.
- Much faster and more memory-efficient, suitable for long time series.

**Cons:** Slightly more complex to understand due to the use of log returns and NumPy operations.

In [45]:
import yfinance as yf
import numpy as np

def remove_worst_n_day_period_fast(df, sv=100, n=11):
    """
    Remove the n-day period whose removal maximises final portfolio value,
    fully accounting for compounding using NumPy for speed.

    Parameters:
        df (DataFrame): Must contain 'Close' column.
        sv (float): Starting portfolio value.
        n (int): Number of consecutive days to remove.

    Returns:
        df_filtered (DataFrame): DataFrame after removing worst block with portfolio value.
        original_total (float): Portfolio value without removing any days.
        new_total (float): Portfolio value after removing worst block.
        worst_block_index (Index): Index of the removed block.
    """
    df = df.copy()
    df["Daily Return"] = df["Close"].pct_change().dropna()
    df = df.dropna()
    daily_log_returns = np.log1p(df["Daily Return"].to_numpy())
    
    # Compute total log return
    total_log_return = np.sum(daily_log_returns)
    
    # Compute rolling n-day sums using cumulative sum (vectorised)
    cumsum = np.cumsum(daily_log_returns)
    rolling_sums = cumsum[n-1:] - np.concatenate(([0], cumsum[:-n]))
    
    # Find the block whose removal maximises final portfolio
    best_start = np.argmax(total_log_return - rolling_sums)
    best_end = best_start + n
    worst_block_index = df.index[best_start:best_end]

    
    # Remove block and recalc portfolio value
    df_filtered = df.drop(worst_block_index)
    df_filtered["Portfolio Value"] = sv * (1 + df_filtered["Daily Return"]).cumprod()
    original_total = sv * (1 + df["Daily Return"]).cumprod().iloc[-1]
    new_total = df_filtered["Portfolio Value"].iloc[-1]
    
    return df_filtered, original_total, new_total, worst_block_index

# Example usage
ticker = yf.Ticker("^SPX")
hd = ticker.history(start="2005-01-01", auto_adjust=True)

hd_filtered, original_total, new_total, removed_block = remove_worst_n_day_period_fast(
    hd, sv=100, n=11
)

print(f"Original total return: ${original_total:.2f}")
print(f"New total return after erasing worst 11-day block: ${new_total:.2f}")
print(f"Worst block removed from {removed_block[0].date()} to {removed_block[-1].date()}")


Original total return: $529.93
New total return after erasing worst 11-day block: $712.59
Worst block removed from 2008-09-26 to 2008-10-10
